In [29]:
# Import libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler, LabelEncoder 
from sklearn.compose import ColumnTransformer
from sklearn import metrics

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## 1- Download and cleaning

1.1- Downlaod

In [30]:
# Reading the dataset
df = pd.read_csv("Walmart_Store_sales.csv")

# Statistique basique
print("Number of rows : {}".format(df.shape[0]))
print()

print("Display of dataset: ")
display(df.head())
print()

print("Basics statistics: ")
data_desc = df.describe(include='all')
display(data_desc)
print()

print("Percentage of missing values: ")
display(100*df.isnull().sum()/df.shape[0])

Number of rows : 150

Display of dataset: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092



Basics statistics: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000



Percentage of missing values: 


Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Holiday_Flag     8.000000
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64

1.2- Cleaning

In [31]:
# Remove NaN in Weekly_Sales
df1 = df.copy(deep=True)
df1 = df1.dropna(subset=['Weekly_Sales'])

display(100*df1.isnull().sum()/df1.shape[0])
display(df1.shape)

Store            0.000000
Date            13.235294
Weekly_Sales     0.000000
Holiday_Flag     8.088235
Temperature     11.029412
Fuel_Price       8.823529
CPI              8.088235
Unemployment    10.294118
dtype: float64

(136, 8)

In [32]:
# Create datetime type columns
df1['Date'] = pd.to_datetime(df1['Date'], format='%d-%m-%Y')
df1.info()

df1['dayofweek'] = df1['Date'].dt.weekday
df1['dayofmonth'] = df1['Date'].dt.day
df1['year'] = df1['Date'].dt.year
df1['month'] = df1['Date'].dt.month

df1.head()

<class 'pandas.core.frame.DataFrame'>
Index: 136 entries, 0 to 149
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Store         136 non-null    float64       
 1   Date          118 non-null    datetime64[ns]
 2   Weekly_Sales  136 non-null    float64       
 3   Holiday_Flag  125 non-null    float64       
 4   Temperature   121 non-null    float64       
 5   Fuel_Price    124 non-null    float64       
 6   CPI           125 non-null    float64       
 7   Unemployment  122 non-null    float64       
dtypes: datetime64[ns](1), float64(7)
memory usage: 9.6 KB


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,dayofweek,dayofmonth,year,month
0,6.0,2011-02-18,1572117.54,NaN,59.61,3.045,214.777523,6.858,4.0,18.0,2011.0,2.0
1,13.0,2011-03-25,1807545.43,0.0,42.38,3.435,128.616064,7.470,4.0,25.0,2011.0,3.0
3,11.0,NaT,1244390.03,0.0,84.57,NaN,214.556497,7.346,NaN,NaN,NaN,NaN
4,6.0,2010-05-28,1644470.66,0.0,78.89,2.759,212.412888,7.092,4.0,28.0,2010.0,5.0
5,4.0,2010-05-28,1857533.70,0.0,NaN,2.756,126.160226,7.896,4.0,28.0,2010.0,5.0


In [33]:
# Remove columns
useless_column = ['Date', 'dayofweek']
df1 = df1.drop(useless_column, axis=1)

In [34]:
# Remove outliers
remove_outlier = ['Temperature','Fuel_Price','CPI','Unemployment']

for col in remove_outlier:
    mean = df1[col].mean()
    std = df1[col].std()

    mask = np.abs((df1[col] - mean) <= 3* std) | ((df1[col] - mean) >= - 3* std)
    df1 = df1[mask]

display(df1.shape)


(94, 10)

## 2-EDA

In [35]:
# Create a bar graph for weekly sales per store 
weekly_sales_mean = df1.groupby(['Store']).agg({'Weekly_Sales':'mean'})

fig = px.bar(weekly_sales_mean, x=weekly_sales_mean.index, y='Weekly_Sales', title="Average weekly sales per store", color='Weekly_Sales')
fig.update_xaxes(title="Store")
fig.update_yaxes(title="Average weekly sales")
fig.show()

The chart above represents the average weekly sales for each store. We can observe that stores 4, 13, 14 and 20 have average sales exceeding 2 million, indicating exceptional performance. This observation could be further explored to understand the successful strategies of these stores.

In [36]:
# Create a bar graph for the average sales when there is a holiday or not in the week
weekly_sales_mean = df1.groupby(['Holiday_Flag']).agg({'Weekly_Sales':'mean'})
fig = px.bar(weekly_sales_mean, x=weekly_sales_mean.index, y='Weekly_Sales', title="Average weekly sales when there is a holiday or not", color='Weekly_Sales')
fig.update_xaxes(title="Holiday_flag")
fig.update_yaxes(title="Average weekly sales")
fig.show()

Although the number of holidays is proportionally lower compared to other days of the year, this graph highlights that average weekly sales remain stable, regardless of the presence of a holiday. The consistency in sales performance is notable, despite the disparity in the distribution of holidays.

In [37]:
# Create a bar graph of the average weekly sales per day in a month
weekly_sales_mean = df1.groupby(['dayofmonth']).agg({'Weekly_Sales':'mean'})
fig = px.bar(weekly_sales_mean, x=weekly_sales_mean.index, y='Weekly_Sales', title="Average weekly sales per day", color='Weekly_Sales')
fig.update_xaxes(title="Day")
fig.update_yaxes(title="Average weekly sales")
fig.show()

This chart shows significant variations in average sales based on the day of the month. Notably, the 8th of the month stands out with an average of sales exceeding 2 million, followed by the 24th and 31st. These specific days could be associated with seasonal trends, recurring promotions, or special buying behaviors. Exploring these days further can help identify strategic opportunities to maximize sales.

In [38]:
# Create a bar graph of the average weekly sale per month
weekly_sales_mean = df1.groupby(['month']).agg({'Weekly_Sales':'mean'})
fig = px.bar(weekly_sales_mean, x=weekly_sales_mean.index, y='Weekly_Sales', title="Average weekly sales per month", color='Weekly_Sales')
fig.update_xaxes(title="Month")
fig.update_yaxes(title="Average weekly sales")
fig.show()

December is, no surprise, the month where average sales are over 2 million.

In [39]:
# Create a bar graph of the average weekly sale per year
weekly_sales_mean = df1.groupby(['year']).agg({'Weekly_Sales':'mean'})
fig = px.bar(weekly_sales_mean, x=weekly_sales_mean.index, y='Weekly_Sales', title="Average weekly sales per year", color='Weekly_Sales')
fig.update_xaxes(title="Year")
fig.update_yaxes(title="Average weekly sales")
fig.show()

The year 2011 was better than the other years. 

In [40]:
# Create a scatter graph: impact of temperature on weekly sales
fig = px.scatter(df1, x='Temperature', y='Weekly_Sales', trendline="ols", title="Weekly sales based on temperature")
fig.show()

We can notice that when temperatures are low, sales are higher. We may be interested in the difference in products sold depending on the time of year or location of stores. 

In [41]:
# Create a scatter graph: impact of fuel prices on weekly sales
fig = px.scatter(df1, x='Fuel_Price', y='Weekly_Sales', trendline="ols", title="Weekly sales based on fuel price")
fig.show()

The chart shows that weekly sales drop when fuel prices are high.

In [42]:
# Create a scatter graph: impact of CPI on weekly sales
fig = px.scatter(df1, x='CPI', y='Weekly_Sales', trendline="ols", title="Weekly sales based on CPI")
fig.show()

The scatter graph highlights a trend where weekly sales are potentially lower when the CPI is high.

In [43]:
# Create a scatter graph: impact of Unemployment on weekly sales
fig = px.scatter(df1, x='Unemployment', y='Weekly_Sales', trendline="ols", title="Weekly sales based on Unemployment")
fig.show()

A low unemployment rate in one sector would tend to lead to higher weekly sales.

In [44]:
# Create a correlation matrix
corr_matrix = df1.corr().round(2)

import plotly.figure_factory as ff

fig = ff.create_annotated_heatmap(corr_matrix.values,
                                  x=corr_matrix.columns.tolist(),
                                  y=corr_matrix.index.tolist()
)

fig.update_layout(title_text='Correlation matrix')

fig.show()

By thoroughly analyzing Walmart’s sales data, some interesting trends emerge. Some stores, such as Stores 4, 13, 14 and 20, show exceptional performance with weekly sales exceeding 2 million. These results could be further explored to identify successful strategies adopted by these stores.

An intriguing observation is the apparent lack of significant impact of holidays on sales. This could influence decisions related to promotions and operations planning during these periods.

Seasonal trends reveal the weeks of 8, 24 and 31 are particularly successful, while the months of December and January are distinguished by the highest sales. 2011 also looks like a good year in terms of average sales.

In terms of the influence of environmental and economic factors on sales, sales appear to decline as temperatures rise, fuel prices rise, or the Consumer Price Index (CPI) is high. On the other hand, a positive correlation is observed with the unemployment rate, suggesting that sales could be more robust in dynamic economic times.

## 3-Model

The exploratory analysis of the data highlighted the significant impact of certain key parameters such as temperature, fuel price, CPI and unemployment rate, as well as time variables such as day of month, mis and year, on sales.

Although all of these factors were considered in the initial analysis, it may be relevant in the later stages to consider a narrower selection of these parameters to simplify the model, improve its interpretability and potentially increase its performance.

For now, we will explore the possibilities with the full feature set, but we will also consider a more targeted approach in our future iterations.

3.1- Preprocessing

In [45]:
# Separate target variable Y from features X
target_name = 'Weekly_Sales'

print("Separating labels from features...")
Y = df1.loc[:,target_name]
X = df1.drop(target_name, axis=1)
print("...Done.")
print(Y.head())
print()
print(X.head())
print()


Separating labels from features...
...Done.
0    1572117.54
1    1807545.43
4    1644470.66
6     695396.19
7    2203523.20
Name: Weekly_Sales, dtype: float64

   Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment  \
0    6.0           NaN        59.61       3.045  214.777523         6.858   
1   13.0           0.0        42.38       3.435  128.616064         7.470   
4    6.0           0.0        78.89       2.759  212.412888         7.092   
6   15.0           0.0        69.80       4.069  134.855161         7.658   
7   20.0           0.0        39.93       3.617  213.023622         6.961   

   dayofmonth    year  month  
0        18.0  2011.0    2.0  
1        25.0  2011.0    3.0  
4        28.0  2010.0    5.0  
6         3.0  2011.0    6.0  
7         3.0  2012.0    2.0  



In [46]:
# Train, test split
print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
print("...Done.")
print()

Dividing into train and test sets...
...Done.



In [47]:
# Create pipeline for numeric features
numeric_features = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'dayofmonth', 'year', 'month']
numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
])

# Create pipeline for categorical features
categorical_features = ['Store', 'Holiday_Flag']
categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore'))
])

# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
])

In [48]:
# Preprocessings on train set
print("Performing preprocessings on train set...")
print(X_train.head())
X_train = preprocessor.fit_transform(X_train)
print('...Done.')
print(X_train[0:5])
print()

# Preprocessings on test set
print("Performing preprocessings on test set...")
print(X_test.head()) 
X_test = preprocessor.transform(X_test)

print('...Done.')
print(X_test[0:5,:])
print()

Performing preprocessings on train set...
    Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment  \
64    4.0           0.0        36.44       3.149  129.898065         5.143   
39    5.0           0.0        60.71       3.297  218.569962         6.300   
62   12.0           0.0        52.77       3.236  126.879484        14.313   
36    3.0           0.0        80.19       3.467  219.741491         7.567   
10    8.0           0.0        82.92       3.554  219.070197         6.425   

    dayofmonth    year  month  
64        16.0  2011.0   12.0  
39        11.0  2011.0   11.0  
62        17.0  2010.0   12.0  
36        23.0  2011.0    9.0  
10        19.0  2011.0    8.0  
...Done.
[[-1.4929771  -0.41143513 -1.04170369 -1.40925613  0.04184723  0.14063251
   1.92858269  0.          0.          1.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0

d:\Programme_D\Miniconda\envs\Daigona\Lib\site-packages\sklearn\preprocessing\_encoders.py:228: UserWarning:

Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros



3.2- Linear Regression

In [49]:
# Train model
print("Train model...")
regressor = LinearRegression()
regressor.fit(X_train, Y_train)
print("...Done.")

# Make predictions
Y_train_pred = regressor.predict(X_train)
Y_test_pred = regressor.predict(X_test)

Train model...
...Done.


In [50]:
# Print R2 score
print("R2 score on training set : ", r2_score(Y_train, Y_train_pred))
print("R2 score on test set : ", r2_score(Y_test, Y_test_pred))

R2 score on training set :  0.9816450826863105
R2 score on test set :  0.9473578763934809


In [51]:
# Cross-validation
scores = cross_val_score(regressor, X_train, Y_train, cv=10)
avg = scores.mean()
std = scores.std()
print("Cross-validation accuracy:", avg)
print("Cross-validation standard deviation:", std)

Cross-validation accuracy: 0.9445320913142135
Cross-validation standard deviation: 0.02814016528446571


Although the model has solid performance on the training set, the difference in performance on the test set suggests some instability in generalization.
We propose to use a regularization method, such as Ridge and Lasso, to improve the robustness of the model and make it more suitable to generalize on new data.

3.3- Features importance

In [52]:
# Features importance
print(regressor.coef_)

[  -49564.89180929   -75905.4042442    662027.67253298   -84369.02896666
   -51149.79119519   -21814.95527375    34387.04018479   170518.40921647
 -1303307.67665164  1945496.41343119 -1342424.44665849    22140.7085293
  -619826.84426965  -746768.58966428 -1259443.39966329  1708326.89755824
  1212194.55663425  1778265.09692492  1003916.78215621   445935.4823868
  -756512.97533723   476135.70543875   860735.78164533  1140004.5702776
   519787.83183498  -167301.97731309]


In [53]:
# Name the features
column_names = []
for name, pipeline, features_list in preprocessor.transformers_:
    if name == 'num': 
        features = features_list 
    else: 
        features = pipeline.named_steps['encoder'].get_feature_names_out() 
    column_names.extend(features) 
        
print("Names of columns corresponding to each coefficient: ", column_names)

Names of columns corresponding to each coefficient:  ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'dayofmonth', 'year', 'month', 'x0_2.0', 'x0_3.0', 'x0_4.0', 'x0_5.0', 'x0_6.0', 'x0_7.0', 'x0_8.0', 'x0_9.0', 'x0_10.0', 'x0_12.0', 'x0_13.0', 'x0_14.0', 'x0_15.0', 'x0_16.0', 'x0_17.0', 'x0_18.0', 'x0_19.0', 'x0_20.0', 'x1_1.0']


In [54]:
# Create a graph bar for the features importance
coefs = pd.DataFrame(index=column_names, data=regressor.coef_.transpose(), columns=["coefficients"])
fig = px.bar(coefs, x = coefs.index, y = "coefficients")
fig.show()

We identified the most important features for our model, including ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'dayofmonth', 'year', 'month', 'Store]. However, it is important to note that while these features have been identified as significant, they have not yet been fully exploited under our current model. A next step could be to specifically integrate these features into the model or explore advanced approaches to take advantage of their importance, which could lead to further improvement in model performance.

## 4- Regularization with Ridge and Lasso

4.1- Ridge

In [61]:
# Performing grid search on Ridge
print("Grid search...")
regressor = Ridge()

params = {
    'alpha': [0.0, 0.1, 0.5, 1.0]
}
linreg_cv = GridSearchCV(regressor, param_grid=params, cv=10)
linreg_cv.fit(X_train, Y_train)
print("...Done.")
print("")
print("Tuned Linear Regression Parameters: {}".format(linreg_cv.best_params_))
print("Best score is {}".format(linreg_cv.best_score_))
print("")
print("R2 score on training set : ", linreg_cv.score(X_train, Y_train))
print("R2 score on test set : ", linreg_cv.score(X_test, Y_test))

Grid search...


...Done.

Tuned Linear Regression Parameters: {'alpha': 0.0}
Best score is 0.9445320913142163

R2 score on training set :  0.9816450826863105
R2 score on test set :  0.9473578763934777


4.2- Lasso

In [58]:
# Performing grid search on Lasso
print("Grid search...")
regressor = Lasso(max_iter=100000)

params = {
    'alpha': [0.001, 0.01, 0.1, 0.5, 1.0, 1.5, 2.5, 5.0, 10.0, 20.0]
}
linreg_cv = GridSearchCV(regressor, param_grid=params, cv=10)
linreg_cv.fit(X_train, Y_train)
print("...Done.")
print("")
print("Tuned Linear Regression Parameters: {}".format(linreg_cv.best_params_))
print("Best score is {}".format(linreg_cv.best_score_))
print("")
print("R2 score on training set : ", linreg_cv.score(X_train, Y_train))
print("R2 score on test set : ", linreg_cv.score(X_test, Y_test))

Grid search...


...Done.

Tuned Linear Regression Parameters: {'alpha': 10.0}
Best score is 0.945324754989087

R2 score on training set :  0.9815591107287789
R2 score on test set :  0.9479540933952665


In conclusion, our comparative analysis between simple linear regression, Ridge and Lasso reveals similar performances, with a slight preference for the Lasso model which seems to generalize slightly better over the test set. However, the difference between the models is minimal, highlighting the overall robustness of the linear approach for prediction in our specific context.